# Check Administrative Levels Based on GPS Coodinates

Many surveys collect GPS coordinates, as well as administrative boundaries like Region, Zone or District. However, admin boundaries can change over time, and/or the surveyor could make a mistake during data entry and mark the boundaries incorrectly. 

As such, there is a need to cross check the GPS coordinates against the official admin boundaries and/or regularly update the data based on new boundaries-or new official shape files from Governments. 

The algorithms below show how to cross check GPS coordinates against geojson files. It requires the following python packages:
1. pandas, 2. numpy and 3. geopandas. 

Although geopandas is realtively easy to install via anaconda, it can be challenging with pip. It requires the installation of Gdal and Fiona in advance of attempting to pip install geopandas. You may need to download the .whl binaries directly. You can download from: https://www.lfd.uci.edu/~gohlke/pythonlibs/


In [1]:
# Imports
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
# import your dataset to check
df = pd.read_csv('gis_amhara_test.csv')

In [3]:
# convert the dataframe into a geopandas dataframe for geospatial analysis. 
# New column created called geometry which contains the point of both lat and lon.
x = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lon,df.lat))

In [4]:
# read in the shape files in geojson format. If you need to change format from shape file to geojson- check out: 
# https://mapshaper.org/ 

gdf_country = gpd.read_file('boundaries/eth_admin0v2.json')
gdf_region = gpd.read_file('boundaries/eth_admin1v2.json')
gdf_zone = gpd.read_file('boundaries/eth_admin2v2.json')

In [5]:
# Function created to iterate through each lat/lon and check if it's within the geojson boundaries. 
# The function takes in two arguments: (1. point(lon, lat), 2. admin boundary dataset)
# The geopandas function .within() is used to check the point against the admin-boundaries and signify 
# Output is the new_g

def check_country(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            country_boolean = gdf['ADM0_PCODE'].iloc[count]
            return country_boolean
        else:
            count +=1

In [6]:
# Another function to check region against GPS. 

def check_region(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_region = gdf['ADM1_PCODE'].iloc[count]
            return new_region # returns actual region based on GPS
        else:
            count +=1

In [7]:
# Another function to check zone against GPS. 

def check_zone(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_zone = gdf['ADM2_PCODE'].iloc[count]
            return new_zone # returns actual zone based on GPS
        else:
            count +=1

In [8]:
# create placeholders for new p_codes in new dataset. 
x['ADM0_PCODE'] = ''
x['ADM1_PCODE'] = ''
x['ADM2_PCODE'] = ''

In [9]:
# Run function on all GPS coordinates in dataset for country

i = 0
count = 0
while i < len(x):
        bb = check_country(x.iloc[i], gdf_country)
        x.loc[i, 'ADM0_PCODE'] = bb
        i +=1

In [10]:
# Run function on all GPS coordinates in dataset for region

i = 0
count = 0
while i < len(x):
        bb = check_region(x.iloc[i], gdf_region)
        x.loc[i, 'ADM1_PCODE'] = bb
        i +=1

In [11]:
# Run function on all GPS coordinates in dataset for zone

i = 0
count = 0
while i < len(x):
        bb = check_zone(x.iloc[i], gdf_zone)
        x.loc[i, 'ADM2_PCODE'] = bb
        i +=1

In [12]:
x.head()

,admin_code,region,zone,name,lat,lon,geometry,ADM0_PCODE,ADM1_PCODE,ADM2_PCODE
0,S0306020712,Amhara,Misrak Gojjam,Tekort,11.0999,37.9873,POINT (37.98730 11.09990),ET,ET03,ET0306
1,S0302010142,Amhara,Debub Gonder,Kahinate Semay,12.2650,38.1583,POINT (38.15830 12.26500),ET,ET03,ET0302
2,S0304090012,Amhara,Debub Wollo,Kedejo 029,11.1348,39.5136,POINT (39.51360 11.13480),ET,ET03,ET0304
3,S0304090772,Amhara,Debub Wollo,Ababale,11.1187,39.4875,POINT (39.48750 11.11870),ET,ET03,ET0304
4,S0304170392,Amhara,Debub Wollo,Yewesa,10.9901,38.6221,POINT (38.62210 10.99010),ET,ET03,ET0304


In [13]:
# Merge datasets basdd on the administrative codes. In this case, it is the PCODE
x= x.merge(gdf_zone[['ADM0_PCODE', 'ADM0_EN', 'ADM1_PCODE', 'ADM1_EN', 'ADM2_PCODE', 'ADM2_EN']], 
             how='inner', on=['ADM0_PCODE','ADM1_PCODE', 'ADM2_PCODE'])

In [14]:
# Review the dataset again and observe the differences between the administrative boundaries. 
# The original region, zone and woreda can be deleted at this stage and replaced with the corrected admin names.
# FYI Woreda = District in Ethiopia.
x.head()

,admin_code,region,zone,name,lat,lon,geometry,ADM0_PCODE,ADM1_PCODE,ADM2_PCODE,ADM0_EN,ADM1_EN,ADM2_EN
0,S0306020712,Amhara,Misrak Gojjam,Tekort,11.099900,37.987300,POINT (37.98730 11.09990),ET,ET03,ET0306,Ethiopia,Amhara,East Gojam
1,S0306040022,Amhara,Misrak Gojjam,Abreha Wo-Atsebeha,10.871799,38.268963,POINT (38.26896 10.87180),ET,ET03,ET0306,Ethiopia,Amhara,East Gojam
2,S0306040252,Amhara,Misrak Gojjam,Mertu Lemariam,10.869573,38.270283,POINT (38.27028 10.86957),ET,ET03,ET0306,Ethiopia,Amhara,East Gojam
3,S0306040142,Amhara,Misrak Gojjam,Bakelaye,10.901465,38.240944,POINT (38.24094 10.90146),ET,ET03,ET0306,Ethiopia,Amhara,East Gojam
4,S0306040522,Amhara,Misrak Gojjam,Debre Gomit,10.912975,38.257915,POINT (38.25792 10.91297),ET,ET03,ET0306,Ethiopia,Amhara,East Gojam


In [15]:
# Write out the CSV for further analysis or cross checking with government counterparts. 
x.to_csv('gis_amhara_test_output.csv', index=True, encoding = 'utf-8')